[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/notebook-share/blob/master/docs/pakistan_floods.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giswqs/notebook-share/HEAD?labpath=docs%pakistan_floods.ipynb)

# Visualization and Analysis of Pakistan Floods 2022 Using Earth Engine and Geemap



## Introduction

From 14 June to October 2022, floods in Pakistan killed 1,739 people, and caused ₨ 3.2 trillion ($14.9 billion) of damage and ₨ 3.3 trillion ($15.2 billion) of economic losses. The flooding was the world's deadliest flood since the 2020 South Asian floods and described as the worst in the country's history. On 25 August, Pakistan declared a state of emergency because of the flooding.

**Sources:**
- Wikipedia: [2022 Pakistan floods](https://en.wikipedia.org/wiki/2022_Pakistan_floods)
- UNICEF: [Devastating floods in Pakistan](https://www.unicef.org/emergencies/devastating-floods-pakistan-2022)


## Installation

In [1]:
# !pip install geemap

## Import libraries

In [2]:
import ee
import geemap.foliumap as geemap

## Create an interactive map

In [3]:
Map = geemap.Map()

country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', 'Pakistan'))

style = {'color': 'red', 'fillColor': '00000000'}
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map.centerObject(country)

Map

## Create Landsat composites

In [4]:
Map = geemap.Map()

col_2021 = ee.ImageCollection('LANDSAT/LC08/C02/T1') \
    .filterDate('2021-06-01', '2021-10-31') \
    .filterBounds(country) \

landsat_2021 = ee.Algorithms.Landsat.simpleComposite(col_2021) \
    .clipToCollection(country)

vis_params = {'bands': ['B6', 'B5', 'B4'], 'max': 128}
Map.addLayer(landsat_2021, vis_params, 'Landsat 2021')


In [5]:
col_2022 = ee.ImageCollection('LANDSAT/LC08/C02/T1') \
    .filterDate('2022-06-01', '2022-10-31') \
    .filterBounds(country) \

landsat_2022 = ee.Algorithms.Landsat.simpleComposite(col_2022) \
    .clipToCollection(country)

Map.addLayer(landsat_2022, vis_params, 'Landsat 2022')
Map.centerObject(country)
Map

## Compare Landsat composites side by side

In [6]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

left_layer = geemap.ee_tile_layer(landsat_2021, vis_params, 'Landsat 2021')
right_layer = geemap.ee_tile_layer(landsat_2022, vis_params, 'Landsat 2022')

Map.split_map(left_layer, right_layer, left_label='Landsat 2021', right_label='Landsat 2022')
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map